<a href="https://colab.research.google.com/github/tpchris1/Gogoro_Routing/blob/master/Gogoro_Routing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import pandas as pd

In [3]:
# Cell for google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# not working since api url is currently unavailable
# data_url='https://wapi.gogoro.com/tw/api/vm/list'
# r = requests.get(data_url,stream = True) #stream=True #開啟時時續續的下載的方式
# r.encoding = 'utf-8'
# print(r.encoding)
# with open('./data.txt','wb') as f: 
#     for chunk in r.iter_content(chunk_size=32): #chunk_size #設定每次下載檔案的大小
#         f.write(chunk)  #每一次迴圈儲存一次下載下來的內容
# f.close

In [13]:
import json
with open("/content/drive/My Drive/Gogoro_Routing/data/raw_data.json", 'r', encoding='utf-8') as f:
    data = json.load(f)
processed_data = []
df = pd.DataFrame(data['data'])
df = df.set_index('Id')
# df.head()
df.iloc[0]


LocName              { List : [{ Value :  Gogoro Huwei Guangfu RS C...
Latitude                                                       23.7112
Longitude                                                      120.434
ZipCode                                                          63244
Address              { List : [{ Value :  No. 478, Guangfu Rd., Huw...
District             { List : [{ Value :  Huwei Township , Lang :  ...
State                                                                1
City                 { List :[{ Value : Yunlin County , Lang : en-U...
AvailableTime                                                     24HR
AvailableTimeByte                                                 None
Name: 865a6d73-ab7d-4176-a542-09898fbf7dca, dtype: object

# 新增區段

# 新增區段

In [14]:
# Change columns order for better view
cols = df.columns.tolist()
cols = [
'LocName',
'Address',
'District',
'City',
'ZipCode',
'Latitude',
'Longitude',
'State',
'AvailableTime',
'AvailableTimeByte']
df = df[cols]
df.head()
# index方法
df.loc['45FC384A-CA7D-43B3-98A7-8A5B635331F5'.lower()]['Latitude']


25.140639

In [8]:
pip install googlemaps

  Created wheel for googlemaps: filename=googlemaps-4.4.2-cp36-none-any.whl size=37858 sha256=b9f154fd9fde2ad748486a76d2d2ab741b67aa0131724ba237304cf4642b1438
  Stored in directory: /root/.cache/pip/wheels/f4/21/41/0c84572e21d52bb322f6c299f38ac7cd8ad6d4d6ce23dc3631
Successfully built googlemaps


In [9]:
# Comparing distance calculate method 
import geopy.distance
import googlemaps
import requests

# find two sample stations -> LocName=['HiLife 八里聖心店站', '新北市民生立體停車場站']
sample_station = ['45FC384A-CA7D-43B3-98A7-8A5B635331F5','C9B55157-153A-4C07-91D5-A3C9131F110D']
sample_station = [i.lower() for i in sample_station]
# ss_index = [df[df['Id'].str.contains(sample_station[0])].index,df[df['Id'].str.contains(sample_station[1])].index]


# Method 1: Lat Long calculate
# coord_0 = (df.loc[sample_station[0]]['Latitude'],df.loc[sample_station[0]]['Longitude'])
# coord_1 = (df.loc[sample_station[1]]['Latitude'],df.loc[sample_station[1]]['Longitude'])
# ll_distance = geopy.distance.distance(coord_0,coord_1).km
# print('ll_distance: ', ll_distance, 'km')

# Method 2: Google distance matrix 
# f = open('confidential/gmap_api_key.txt',encoding="utf-8")
# gmaps_key = f.readline() 
# url = 'https://maps.googleapis.com/maps/api/distancematrix/json?'
# url += 'origins='+str(coord_0[0])+','+str(coord_0[1])
# url += '&destinations='+str(coord_1[0])+','+str(coord_1[1])
# url += '&mode=driving'
# url += '&avoid=tolls|highways|ferries'
# url += '&key='+gmaps_key
# url
# resp = requests.get(url)
# print('gmap_distance:',resp.json()['rows'][0]['elements'][0]['distance']['text'])

# gmaps api沒辦法支持多avoid
# gmaps = googlemaps.Client(key=gmaps_key)
# gmaps.distance_matrix(origins=str(coord_0[0])+','+str(coord_0[1]),destinations=str(coord_1[0])+','+str(coord_1[1]), mode='driving', avoid='tolls|highways')


In [10]:
# Construct ll_matrix
size = len(df)
ll_df = pd.DataFrame(index=range(size),columns=range(size))

#建一個站點名的list
lst = []
for i in range(size):
    lst.append(df.index[i])

ll_df.index = list(lst)
ll_df.columns = lst
# print(df.index[0])
for i in range(size):
    for j in range(i+1,size):
        station1 = df.index[i]
        station2 = df.index[j]
        coord_0 = (df.loc[station1]['Latitude'],df.loc[station1]['Longitude'])
        coord_1 = (df.loc[station2]['Latitude'],df.loc[station2]['Longitude'])
        ll_distance = geopy.distance.distance(coord_0,coord_1).km
        ll_df.loc[station1, station2] = ll_distance

# output ll_df to .csv
ll_df.to_csv(r'/content/drive/My Drive/Gogoro_Routing/data/ll_matrix.csv')
ll_df.head()


,865a6d73-ab7d-4176-a542-09898fbf7dca,cc480eeb-93c4-49b1-9b12-15474249b5bf,5a72a2fb-505c-476b-a796-17fcf769c99a,41c44a19-d494-4845-a659-19d5cba5b29a,5dfa7fd8-2148-4fb2-b3b3-2d98949c0227,72448e88-d147-4640-8e72-2f5f85240484,7ae3c1f2-c250-4032-ac8c-30b76d37aea0,6ae3b11f-dee5-4944-9c0a-310a956c3e9c,911d709e-a0c6-4f07-ad76-364cfa8091e6,99c0fa96-2d0a-41e2-8d45-3bac8afdfb91,5aa573ac-9749-4b44-96d9-45c011271bff,24d2d276-80fb-44ff-83af-49003d3527ec,7d5915e0-c312-4c5d-98f0-49448f23c7cd,cc7c84a5-f7ce-412e-8ca8-52c2fa4277e6,10e4ebb6-6428-47ae-85df-6266dabca45f,3e742441-cd80-419f-923b-637ddb26dc30,20839915-f8ce-48ba-9b30-6dad16baf33e,766c02fe-2556-4d5a-ae40-6fa8a07de439,9aebcaf8-4416-4735-9f88-79acf059b9c1,5fd41ebc-d4fd-4a54-974e-88539d6d4753,9378da9a-e89b-4b69-aa85-9430ca8f9d15,36ba5441-6d4b-4880-b2c8-a228279d33f2,7428bf0b-d4c8-4b25-aa12-b05d587db155,578be395-f2e2-4d88-a20e-b99f0e5c9dda,38eb9036-e4ee-4b1c-bd06-bc87b40133e7,6538b6c2-7caa-4b37-8363-c3610c992912,7bb72ef8-3753-4e07-adda-cbdbba3e905a,7b72977f-9cf0-4865-9c17-cdaaad3d28f4,dc2a8557-4fd4-48c0-9e35-d84d086cbf8e,ae797761-0964-4bf4-bb60-e8ec9c800f77,ebd52345-65dc-4f38-acd4-f1db75b30f9b,b765f85e-6cae-478b-9927-f328fa5ef1bf,7028978e-7379-4391-938b-f7c73b0d1885,c900d67d-84e1-4b9e-907b-f7cb2dc2c379,66f06138-d475-40d2-bd09-f7e6f33559f5,26a7b63d-2caf-4cac-ba1c-f833b4814236,de1cb98b-ed92-42a5-a18a-f9a19a14686f,55016a40-76b2-423b-9255-f62d90441a05,10ce873c-b233-4556-9caf-f66ccf0b207d,669ece4f-29f4-46b2-a542-f1ddcf64cffa,...,4606025a-9a5f-4076-aab7-24289df31af6,787d37a2-260e-4362-ae01-20d93f6a0df0,8f1ac09f-eb7f-41a1-aa02-202c63dd4dc9,14417361-35f6-40ce-974d-205a68f02d7f,3d2fcce8-0912-4cc3-bb3f-1fc00043bc07,b8e4632d-8d15-463b-a7bf-1fcdf41ba0d5,0f4f9a7f-06a1-462d-9372-22607ca69fb0,d707bfbb-7272-40bf-b711-17c16f80f6a2,6f029be5-ee81-4518-b0e5-1835f4ea75fc,8778f7c6-a6b8-43cb-9e9e-1bbb959c2ff3,e9a226f9-8db4-4247-8c58-1b5f62f8d680,78c4a21b-20e8-463e-bbf3-1c91bb53ca65,d3f5dfc4-052b-4075-9058-1dcc41b86318,aefb3d9e-8dde-4f62-8adc-1ef30ec9b353,2c1217b5-e4e3-4c2d-a5ba-1f348b309d36,a9208868-3ad7-4e07-ae37-1f48f50b5526,e4c74ccf-3acf-4cb8-a1bf-13a1fa14d80a,b0ee3d34-dba0-4d56-a521-16a4d91ef5e0,a130243f-2101-41b5-a344-1628a8f68f45,c914d8f0-7bfe-4cd6-b9dc-15958ed8f9de,eb055cf5-0dd8-4f83-a874-159eeaee9bd5,4118232e-7972-44c0-ae1d-081d6b924b16,6993002f-f99c-4f5c-9858-08cf15d4e0e9,6e0265ce-7cd6-46ed-a0c8-0887a20dc864,6e956747-a069-47ed-9105-0aa464e556bb,1697827c-d633-43b9-9d14-0b7068d33282,2dd85045-4247-4999-8189-0b750811d91d,75736a51-6399-42ba-96a1-0bb84e8bbb24,64485ca8-fb46-4052-8344-0fd3389273cd,1e1ab7c8-8483-4c41-a147-0ea90647a829,7b2f1fe3-3d22-4f6c-9094-0f81fcb721fa,ae701647-65aa-4d77-9890-0f83d874cfd4,0f949ea3-7352-49a1-92fd-01b3c386e6ee,f2e2ed67-712a-437a-a908-012e553cd12d,2608ebf6-f1db-4d24-b3f1-03590e4d44d6,b40c5eb7-7606-4b93-87ca-02fe58da5838,f3a88903-7cc3-482c-856f-02a23ca5b87b,16a11557-2db4-4fb9-a6d0-06bde48315fd,b9540af6-960a-4ace-9897-0418f752dced,b407e167-2fa1-4a42-8aa2-040677b460bf
865a6d73-ab7d-4176-a542-09898fbf7dca,NaN,11.9307,9.86579,5.46833,0.479106,18.7893,9.65724,29.1522,10.3315,20.8384,15.1433,7.12498,23.6127,9.91895,8.4469,0.507137,14.5365,10.3516,9.33526,3.1888,10.5683,4.76837,1.11979,18.593,9.63695,8.7649,12.4794,6.06729,10.7012,20.2673,1.61085,0.975779,6.25864,103.858,108.498,120.238,116.321,125.456,121.653,126.789,...,81.0269,79.4546,75.2226,76.9119,78.9262,78.7566,83.9783,84.3608,80.5676,82.7312,76.3608,82.3066,82.327,79.9451,84.8176,63.5225,83.3362,77.9253,84.187,77.9136,81.7461,83.7661,82.9064,62.857,69.1739,84.6144,83.3153,81.3233,81.4068,83.8546,65.9875,83.891,75.4583,78.6605,76.8923,80.7997,83.4732,84.0307,83.5728,70.9973
cc480eeb-93c4-49b1-9b12-15474249b5bf,NaN,NaN,2.43909,16.268,12.178,30.0069,13.4173,39.6706,2.01092,29.6257,3.85968,15.6359,35.4821,2.04166,10.7838,11.6675,5.99555,1.79613,3.72677,14.1846,1.39609,9.36898,12.8902,8.14597,2.35974,7.47192,0.689858,8.36461,1.397,29.3035,10.3461,12.6149,7.87409,103.706,109.504,121.528,117.534,126.626,122.148,127.219,...,84.2963,83.10

In [14]:
def GoogleMap_Distance(station1_latitude,station1_longitude,station2_latitude,station2_longitude):
  f = open('/content/drive/My Drive/Gogoro_Routing/confidential/gmap_api_key.txt',encoding="utf-8")
  gmaps_key = f.readline() 
  url = 'https://maps.googleapis.com/maps/api/distancematrix/json?'
  url += 'origins='+str(station1_latitude)+','+str(station1_longitude)
  url += '&destinations='+str(station2_latitude)+','+str(station2_longitude)
  url += '&mode=driving'
  url += '&avoid=tolls|highways|ferries'
  url += '&key='+gmaps_key
  url
  resp = requests.get(url)
  print('gmap_distance:',resp.json()['rows'][0]['elements'][0]['distance']['text'])
  return resp.json()['rows'][0]['elements'][0]['distance']['text']

In [25]:
def Multi_GoogleMap_Distance(station1_latitude,station1_longitude,list_of_ll):
  f = open('/content/drive/My Drive/Gogoro_Routing/confidential/gmap_api_key.txt',encoding="utf-8")
  gmaps_key = f.readline() 
  url = 'https://maps.googleapis.com/maps/api/distancematrix/json?'
  url += 'origins='+str(station1_latitude)+','+str(station1_longitude)
  url += '&destinations='+ list_of_ll
  url += '&mode=driving'
  url += '&avoid=tolls|highways|ferries'
  url += '&key='+gmaps_key
  url
  resp = requests.get(url)
  #print('gmap_distance:',resp.json()['rows'][0]['elements'][0]['distance']['text'])
  return resp.json()

In [11]:
# Construct adjacent graph for stations
import pandas as pd
ll_df = pd.read_csv('/content/drive/My Drive/Gogoro_Routing/data/ll_matrix.csv',index_col = 0)
station_df = pd.read_csv()
ll_df.columns[0]
size = len(ll_df)
gMap_df = pd.DataFrame(index=range(size),columns=range(size))
#建一個站點名的list
lst = []
for i in range(size):
    lst.append(ll_df.index[i])
gMap_df.index = list(lst)
gMap_df.columns = lst

#ll_df.head()
for i in range(1):
    station1 = ll_df.index[i]
    stations_25= []
    for j in range(i+1,26):
        station2 = ll_df.index[j]
        if ll_df.loc[station1,station2] < 80:
            stations_25.append(station2)
            if len(stations_25) == 25:
                print(stations_25)
                # send api
                
                stations_25.clear()


# ll_df.loc[ll_df.index[0],ll_df.index[1]]
# for i in range(size):
#     station1 = ll_df.index[i]
#     for j in range(i+1,size):
#         station2 = ll_df.index[j]
#         if ll_df.loc[station1,station2] < 80:
#             gMap_df.loc[station1,station2] = GoogleMap_Distance(df.loc[station1]['Latitude'],df.loc[station1]['Longitude'],df.loc[station2]['Latitude'],df.loc[station2]['Longitude'])

# output ll_df to .csv
# gMap_df.to_csv(r'/content/drive/My Drive/Gogoro_Routing/data/gMap_df.csv')
# gMap_df.head()

['cc480eeb-93c4-49b1-9b12-15474249b5bf', '5a72a2fb-505c-476b-a796-17fcf769c99a', '41c44a19-d494-4845-a659-19d5cba5b29a', '5dfa7fd8-2148-4fb2-b3b3-2d98949c0227', '72448e88-d147-4640-8e72-2f5f85240484', '7ae3c1f2-c250-4032-ac8c-30b76d37aea0', '6ae3b11f-dee5-4944-9c0a-310a956c3e9c', '911d709e-a0c6-4f07-ad76-364cfa8091e6', '99c0fa96-2d0a-41e2-8d45-3bac8afdfb91', '5aa573ac-9749-4b44-96d9-45c011271bff', '24d2d276-80fb-44ff-83af-49003d3527ec', '7d5915e0-c312-4c5d-98f0-49448f23c7cd', 'cc7c84a5-f7ce-412e-8ca8-52c2fa4277e6', '10e4ebb6-6428-47ae-85df-6266dabca45f', '3e742441-cd80-419f-923b-637ddb26dc30', '20839915-f8ce-48ba-9b30-6dad16baf33e', '766c02fe-2556-4d5a-ae40-6fa8a07de439', '9aebcaf8-4416-4735-9f88-79acf059b9c1', '5fd41ebc-d4fd-4a54-974e-88539d6d4753', '9378da9a-e89b-4b69-aa85-9430ca8f9d15', '36ba5441-6d4b-4880-b2c8-a228279d33f2', '7428bf0b-d4c8-4b25-aa12-b05d587db155', '578be395-f2e2-4d88-a20e-b99f0e5c9dda', '38eb9036-e4ee-4b1c-bd06-bc87b40133e7', '6538b6c2-7caa-4b37-8363-c3610c992912']

In [33]:
import requests
temp_list = ['cc480eeb-93c4-49b1-9b12-15474249b5bf', '5a72a2fb-505c-476b-a796-17fcf769c99a', '41c44a19-d494-4845-a659-19d5cba5b29a', '5dfa7fd8-2148-4fb2-b3b3-2d98949c0227', '72448e88-d147-4640-8e72-2f5f85240484', '7ae3c1f2-c250-4032-ac8c-30b76d37aea0', '6ae3b11f-dee5-4944-9c0a-310a956c3e9c', '911d709e-a0c6-4f07-ad76-364cfa8091e6', '99c0fa96-2d0a-41e2-8d45-3bac8afdfb91', '5aa573ac-9749-4b44-96d9-45c011271bff', '24d2d276-80fb-44ff-83af-49003d3527ec', '7d5915e0-c312-4c5d-98f0-49448f23c7cd', 'cc7c84a5-f7ce-412e-8ca8-52c2fa4277e6', '10e4ebb6-6428-47ae-85df-6266dabca45f', '3e742441-cd80-419f-923b-637ddb26dc30', '20839915-f8ce-48ba-9b30-6dad16baf33e', '766c02fe-2556-4d5a-ae40-6fa8a07de439', '9aebcaf8-4416-4735-9f88-79acf059b9c1', '5fd41ebc-d4fd-4a54-974e-88539d6d4753', '9378da9a-e89b-4b69-aa85-9430ca8f9d15', '36ba5441-6d4b-4880-b2c8-a228279d33f2', '7428bf0b-d4c8-4b25-aa12-b05d587db155', '578be395-f2e2-4d88-a20e-b99f0e5c9dda', '38eb9036-e4ee-4b1c-bd06-bc87b40133e7', '6538b6c2-7caa-4b37-8363-c3610c992912']
temp_str = ''
for i in temp_list:
    lat = str(df.loc[i]['Latitude'])
    longt = str(df.loc[i]['Longitude'])
    tmp_str = lat + ',' + longt
    temp_str += tmp_str + '|'
temp_str = temp_str[:-1]
temp_str
lati = df.loc[df.index[0]]['Latitude']
longt = df.loc[df.index[0]]['Longitude']
json = Multi_GoogleMap_Distance(lati,longt,temp_str)
json

{'destination_addresses': ['No. 43號, Zhuangjing Road, Douliu City, Yunlin County, Taiwan 640',
  'Zhengxin High School, Douliu City, Yunlin County, Taiwan 640',
  'No. 131號, Jianguo Road, Tuku Township, Yunlin County, Taiwan 633',
  'No. 23號, Section 2, Linsen Road, Huwei Township, Yunlin County, Taiwan 632',
  '638, Taiwan, Yunlin County, Mailiao Township, 雲林縣麥寮鄉橋頭村橋頭160號',
  'No. 56, Xingnong West Road, Xiluo Township, Yunlin County, Taiwan 648',
  'No. 99號, Section 1, Zhongzheng Road, Kouhu Township, Yunlin County, Taiwan 653',
  'No. 350, Section 2, Mingde North Road, Douliu City, Yunlin County, Taiwan 640',
  'No. 66, Huanan Road, Beigang Township, Yunlin County, Taiwan 651',
  '640, Taiwan, Yunlin County, Douliu City, 雲55鄉道30號',
  'No. 102, Section 1, Zhongshan Road, Erlun Township, Yunlin County, Taiwan 649',
  'No. 36, Jhongshan Rd, Taixi Township, Yunlin County, Taiwan 636',
  'No. 297號, Section 2, Yunlin Road, Douliu City, Yunlin County, Taiwan 640',
  '647, Taiwan, Yunlin Co

In [35]:
json['rows'][0]['elements'][1]['distance']['text']

'12.4 km'